## RAG implementation

### Preprocess data

In [1]:
import pandas as pd
from tqdm import tqdm

def generate_UVQuAD(df: pd.DataFrame, output_path: str = "UVQuAD.csv"):
    """Generate UVQuAD from data_raw.csv"""
    new_data = []
    for i in tqdm(range(len(df))):
        request = eval(df["pergunta"][i])
        for p in request["perguntas"]:
            # print(request["perguntas"][p])
            # print(type(request["respostas"][p]))
            new_data.append(
                {
                    "title": df["title"][i],
                    "context": df["context"][i],
                    "question": request["perguntas"][p],
                    "answer": {
                        "text": list(request["respostas"][p].values()),
                        "answer_start": [],},
                }
            )
    new_df = pd.DataFrame(new_data)
    new_df.to_csv(output_path, index=False)
    return new_df

# Load data_raw
df = pd.read_csv('../data/processed/data_raw.csv')

# Generate UVQuAD
df = generate_UVQuAD(df, "../data/processed/UVQuAD.csv")

100%|██████████| 422/422 [00:00<00:00, 7472.37it/s]


### Retriever Probe

In [2]:
!pip install rank_bm25

In [3]:
from langchain.retrievers import BM25Retriever


In [8]:
df = pd.read_csv('../data/processed/contextos.csv')
df.head(1)

,title,context
0,titulo,Vestibular Unicamp 2024\nResolução GR-031/2023...


In [9]:
retriever = BM25Retriever.from_texts(df['context'].to_list())

In [10]:
result = retriever.get_relevant_documents("Quantas vagas regulares são oferecidas para ingresso nos Cursos de Graduação da Unicamp em 2024?")
result

[Document(page_content='Art. 1º Para o ano de 2024 são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp distribuídas nos seguintes sistemas de ingresso:\xa0 I. 2537 vagas oferecidas pelo Vestibular Unicamp (VU) 2024.\xa0 II. 314 vagas oferecidas pelo Edital ENEM-Unicamp 2024.\xa0 III. 325 vagas oferecidas pelo Provão Paulista 2024.\xa0 IV. 49 vagas oferecidas pelo Vestibular Indígena (VI) 2024. O Vestibular Indígena terá ainda 81 vagas adicionais, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0 IV. 115 vagas oferecidas pelo Edital de olimpíadas cientíﬁcas e competições de conhecimento de áreas especíﬁcas. Haverá, ainda, 14 vagas adicionais nesse sistema de ingresso, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0'),
 Document(page_content='§1º As vagas regulares não preenchidas nos incisos II, III, IV e V serão transferidas para o VU 2024 para que sejam preenc

### Retriever Class

In [112]:
from langchain.retrievers import BM25Retriever
import pandas as pd

class ConvHistory:
    def __init__(self):
        self.messages = []
        self.offset = 0
        self.system_message = "¡Hola! Soy tu chatbot para el Vestibular da Unicamp 2024. ¿En qué puedo ayudarte?"
        
    def set_system_message(self, system_message: str):
        """Set the system message."""
        self.system_message = system_message

    def append_message(self, role: str, message: str):
        """Append a new message."""
        self.messages.append([role, message])
    
    def to_openai_api_messages(self):
        """Convert the conversation to OpenAI chat completion format."""
        ret = [{"role": "system", "content": self.system_message}]

        for i, (_, msg) in enumerate(self.messages[self.offset :]):
            if i % 2 == 0:
                ret.append({"role": "user", "content": msg})
            else:
                if msg is not None:
                    ret.append({"role": "assistant", "content": msg})
        return ret

class GPT3ChatBot:
    def __init__(self):
        self.model = "gpt-3.5-turbo"
        self.agent = OpenAI(
            api_key= os.getenv("OPENAI_API_KEY"),
        )
        self.retriever = UVQuADChatBotRetriever()
        
    def get_answer(self, question: str):
        relevant_documents = self.retriever.get_relevant_documents(question)
        return relevant_documents[0]
    
    def get_answer_from_user_input(self, user_input: str):
        return self.get_answer(user_input)
    
    def get_answer_from_conversation(self, conversation: list):
        return self.get_answer(conversation)
    
    def get_answer_from_gpt3(self, question: str):
        response = self.agent.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "Vocé é um chatbot para o Vestibular da Unicamp 2024."},
            ] + question,
        )
        return response.choices[0].message.content

class UVQuADChatBotRetriever:
    def __init__(self):
        self.data = pd.read_csv('../data/processed/contextos.csv')
        self.from_texts(self.data['context'].to_list())
        
    def from_texts(self, texts: list):
        self.retriever = BM25Retriever.from_texts(texts)
        
    def get_relevant_documents(self, question: str, n_docs: int = 1):
        result = self.retriever.get_relevant_documents(question)
        return result
    
# class UVQuADChatBotRetriever:
#     def __init__(self):
#         self.data = pd.read_csv('../data/processed/contextos.csv')
#         self.from_texts(self.data['context'].to_list())
        
#     def from_texts(self, texts: list):
#         self.retriever = BM25Retriever.from_texts(texts)
        
#     def get_relevant_documents(self, question: str, n_docs: int = 1):
#         result = self.retriever.get_relevant_documents(question)
#         return result

In [111]:
q = "Quantas vagas regulares são oferecidas para ingresso nos Cursos de Graduação da Unicamp em 2024?"
retriever = UVQuADChatBotRetriever()
retriever.get_relevant_documents(q)

[Document(page_content='Art. 1º Para o ano de 2024 são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp distribuídas nos seguintes sistemas de ingresso:\xa0 I. 2537 vagas oferecidas pelo Vestibular Unicamp (VU) 2024.\xa0 II. 314 vagas oferecidas pelo Edital ENEM-Unicamp 2024.\xa0 III. 325 vagas oferecidas pelo Provão Paulista 2024.\xa0 IV. 49 vagas oferecidas pelo Vestibular Indígena (VI) 2024. O Vestibular Indígena terá ainda 81 vagas adicionais, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0 IV. 115 vagas oferecidas pelo Edital de olimpíadas cientíﬁcas e competições de conhecimento de áreas especíﬁcas. Haverá, ainda, 14 vagas adicionais nesse sistema de ingresso, conforme Edital a ser publicado, respeitando os princípios da Deliberação CONSU-A-032/2017.\xa0')]